
## Informações Gerais
| Informações |  Detalhes  |
|-------------|------------|
|Origem | https://stbrewerydatalake.blob.core.windows.net/brewery-project/0.landing_zone/brewery_raw |
|Nome Tabela  | bronze.brewery |

## Histórico de Atualizações
| Data | Desenvolvido por | Motivo |
|:----:|------------------|--------|
|19/04/2024 | Gabriel Alves | Criação do notebook |


In [0]:
import datetime
from pyspark.sql.functions import *
from delta.tables import DeltaTable

In [0]:
# Definição de variáveis de configuração
storage_account        = "stbrewerydatalake"       # sua Storage Account
container_name         = "brewery-project"         # container ADLS Gen2
landing_zone_container = "0.landing_zone"         # pasta da camada Bronze
bronze_container       = "1.bronze"                # pasta final do Bronze em Delta
name_file              = "brewery_raw"
database               = "bronze"
tabela                 = "brewery"

# Paths de entrada e saída
input_path = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/{landing_zone_container}/{name_file}.json"
bronze_delta_path = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/{bronze_container}"

In [0]:
# Inclusão de timestamp
df_brewery = spark.read.json(input_path)
df_brewery = df_brewery.withColumn("ingest_timestamp", current_timestamp())

In [0]:
# Criar o database bronze se não existir
spark.sql(f"CREATE DATABASE IF NOT EXISTS {database}")

# Salvar dados como tabela Delta externa no metastore, gravando arquivos físicos na pasta Bronze
df_brewery.write \
        .format('delta') \
        .mode('overwrite') \
        .option('overwriteSchema', 'true') \
        .option('path', bronze_delta_path) \
        .saveAsTable(f'{database}.{tabela}')


# (Opcional) Verificação visual do diretório Bronze Delta
display(dbutils.fs.ls(bronze_delta_path))

path,name,size,modificationTime
abfss://brewery-project@stbrewerydatalake.dfs.core.windows.net/1.bronze/_delta_log/,_delta_log/,0,1747698138000
abfss://brewery-project@stbrewerydatalake.dfs.core.windows.net/1.bronze/part-00000-bb46f8cd-0bd1-4317-a7bd-1a8ff11b18a6.c000.snappy.parquet,part-00000-bb46f8cd-0bd1-4317-a7bd-1a8ff11b18a6.c000.snappy.parquet,12214,1747698147000
